Проверочная #9 (по спектральной кластеризации) - https://goo.gl/forms/1TCba95inBw36Jfa2

# Анализ графов и социальных сетей

Нам понадобится библиотека igrpah - http://www.lfd.uci.edu/~gohlke/pythonlibs/#python-igraph

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import Image, display

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

Рассмотрим рассчеты мер центральности для сети Zachary Karate Club.

Данная сеть имеет довольно [живую историю](https://en.wikipedia.org/wiki/Zachary's_karate_club) и её часто используют для демонстрации некоторых концептов анализа социальных сетей (SNA).

Напомним обозначения:
* $A$ - матрица смежности
* $A_{ij}$ - элемент на $i$-й строке и $j$-м столбце
* $v_i$ - $i$-я вершина
* $k(i)$ - степень вершины $v_i$
* $d(i, j)$ - расстояние между вершинами $v_i$ и $v_j$

Основная задача анализа графов и социальных сетей - анализ важности вершин. Существует несколько подходов к решению этой задачи:
* степень центральности (число инцедентных вершин) - degree centrality,
* степень близости (количество наименьших путей) - closeness centrality,
* степень посредничества (контроль кратчайших путей) - betweenness centrality,
* степень влиятельности (связь с другими влиятельными вершинами) - eidgenvector centrality, page rank.

In [ ]:
# Функция для рисования графов с лейблами
def plot_graph(g, settings=dict()):
    ig.plot(g, 'graph.png', **settings)
    display(Image(filename='graph.png'))

Изобразим сеть без акцентов на важность вершин (в каком бы то ни было смысле).

Напомним себе, что люди под номерами `1` и `34` сыграли ключевую роль в дальнейшем распаде клуба.

In [ ]:
g = ig.Graph.Famous('Zachary')
graph_layout = g.layout_fruchterman_reingold()

vision_settings = dict()
vision_settings['layout'] = graph_layout
vision_settings['vertex_label'] = range(1,35)

In [ ]:
plot_graph(g, settings=vision_settings)

# Degree centrality

Самая оцевидная центральность - просто степень узла. Характеризует некоторую популярность узла (много друзей, много связей).

$$ C_d(i) = k(i) = \sum_jA_{ij} = \sum_iA_{ij}$$
$$ \bar{C}_d(i) = \frac{1}{n-1} C_d(i)$$

Существует обобщение на ориентированные (prestige) и взвешенные сети.

In [ ]:
degr = g.degree()
vision_settings['vertex_size'] = np.array(degr) * 2

plot_graph(g, settings=vision_settings)

# Closeness centrality

Центральность, основанная на расстоянии до остальных вершин в графе.

$$ C_{cl}(i) = \frac{1}{\sum_j d(i,j)} $$

$$ \bar{C}_{cl}(i) = (n-1) \cdot C_{cl}(i) $$

Актор, расположенный в центре сети может быстро добраться до остальных акторов. Акторы на периферии расположены дальше.

**Вопрос:** что будет, если граф окажется несвязным?

In [ ]:
close = g.closeness(normalized=True)
vision_settings['vertex_size'] = np.array(close) * 50

plot_graph(g, settings=vision_settings)

# Betweenness centrality (nodes)

Пусть $\sigma_{st}$ - количество кратчайших путей между вершинами $s$ и $t$, а $\sigma_{st}(i)$ - кр. пути между $v_s$ и $v_t$, которые проходят через вершину $v_i$.

Тогда 
$$ C_b(i) = \sum\limits_{s\neq t\neq i} \frac{\sigma_{st}(i)}{\sigma_{st}} $$

$$ \bar{C}_b(i) = \frac{2}{(n-1)(n-2)}C_b(i) $$

In [ ]:
betw = np.array(g.betweenness(directed=False))
vision_settings['vertex_size'] = betw / 2 

plot_graph(g, settings=vision_settings)

# Betweenness centrality (edges)

#### Задание
Betweenness также можно расчитывать для ребер! Давайте определим для каких ребер она наибольшая и что это может нам дать?

In [ ]:
betw_edg = g.edge_betweenness(directed=False)
# Your code here

# Eigenvector centrality

Идея Eigenvector centrality в том, что "важность" текущей вершины должна зависеть от важности соседней с ней вершин

$$ v_i = \frac{1}{\lambda}\sum_j A_{ij} v_j $$
* $v_i$ - важность $i$-й вершины

Можно переписать в матричный вид:
$$Av = \lambda v$$
А это задача о собственном числе матрицы $A$! 

$$C_{eig} = v_1,$$
где $v_1$ - собственный вектор при наибольшем собственном числе $\lambda_1$

Рассмотрим простой граф и его матрицу смежности:

<img src="../figures/pagerank1.png" alt="Simple matrix" style="width: 500px;"/>

Воспользуемся самым простым вариантом итеративного вычисления важности вершин:

<img src="../figures/pagerank2.png" alt="Presige" style="width: 500px;"/>

In [ ]:
# Реалзуем этот алгоритм

def prestige(A, e=0.001):
    n = A.shape[0]
    p = np.ones(n)
    lam = 0
    # ваш код здесь
    return p, lam

Посчитаем его для простого графа и построим график сходимости собственного значения $\lambda$

In [ ]:
# Ваш код здесь

А теперь воспользуемся встроенной функцией в igraph

In [ ]:
# Ваш код здесь

# Page Rank

Идея PageRank заключается в попытке описать блуждание по вершинам графа. Вероятность перехода в вершину $v_i$ обратнопропорциональна степеням входящих связанных с ней вершин.

$$p^{t+1} = (D^{-1}A)^\top p^t = P^\top p^t$$

Помимо случайного блуждания между соседними вершинами заложен механизм "телепорта" между случайными вершинами с вероятностью $1-\alpha$.

$$ \mathbb{P} = \alpha P + \frac{(1 - \alpha)}{n} E,$$
где $E$ - это матрица состоящая из единиц.

Аналогичным образом решается задача на поиск собственного числа

$$\mathbb{P}^\top p = \lambda p$$

In [ ]:
pr = g.pagerank(directed=False)
vision_settings['vertex_size'] = np.array(pr) * 500

plot_graph(g, settings=vision_settings)

#### Задание 
Сравните PageRank, EigenVector Centrality и Degree Centrality для данного графа. Какие выводы можно сделать?

In [ ]:
# Your code here

# "Геометрическая" центральность

Eccentricity - максимальная длина кратчайшего пути из вершины $i$ до всех остальных вершин $e(i) = \max\limits_j d(i, j)$.

Диаметр - $\max e(i)$<br/>
Радиус - $\min e(i)$

Центральными вершинами являются те, у которых $e(i)$ равна радиусу графа

In [ ]:
print(g.radius())
print(g.diameter())

In [ ]:
ecc = g.eccentricity()
vision_settings['vertex_size'] = np.array(ecc) * 10

plot_graph(g, settings=vision_settings)

# Clustering coefficient

Доля "треугольников" в окресности вершины.

In [ ]:
clust = g.transitivity_local_undirected()
vision_settings['vertex_size'] = np.array(clust) * 50
 
plot_graph(g, settings=vision_settings)

In [ ]:
avg_clust = g.transitivity_avglocal_undirected()
clust_coeff = g.transitivity_undirected()

print('Средний коэффициент кластеризации: %f' % avg_clust)
print('Глобальный коэффициент кластеризации: %f' % clust_coeff)

# Структура сети без визуализации

#### Задание

На одном графике изобразите Closeness и Betwenneess Centalities. Как визуализация соотносятся с положением вершин в графе?

In [ ]:
## Your Code Here

# HITS + PageRank

####  Задание

Для неориентированного графа PageRank, Eigenvector, а уж темболее HITS не имеют особого смысла.

Рассмотрите граф связи политических блогов наканануне выборов в США (data/graph/polblogs.gml). Связь между вершинами имеет место если в постах одного блога были ссылки на другой блог.

* Проанализируйте структуру графа и почистите его перед расчетом центральностей
* Посчитайте PageRank и HITS
* Найдите топовые вершины по этим метрикам и изобразите их окресность

In [ ]:
## Your Code Here

# Социальный граф

Пример социального графа в "сыром виде" лежит в папке data/graph

Один `json` файл содержит простое описание друзей, второй списки общих друзей с вашими непосредственными друзьями. Остается только преобразовать эту информацию для работы с igpraph ( или другим ПО для работы с сетями )

* Загрузите данные в тетрадку и преобразуйте их в граф
* Сохраните граф в формат GML для дальнейшей работы
* Посчитайте центральности и выясните, кто в вашем графе другей самый.. ээ.. центральный =)